In [1]:
# carregar bibliotecas

from samgeo.text_sam import LangSAM
import glob
import os

In [2]:
# escolha do modelo
# vit_b(91M), vit_l(308M), vit_h(636M)

sam = LangSAM(model_type='vit_b')

final text_encoder_type: bert-base-uncased


In [3]:
def processar_imagens(caminho_pasta, texto_prompt, box_t, text_t, pasta_saida):
  """
  Processa todas as imagens de uma pasta.

  Args:
    caminho_pasta: Caminho da pasta que contém as imagens.
    texto_prompt: Prompt de texto para a predição.
    box_t: Threshold para a detecção de caixas delimitadoras.
    text_t: Threshold para a detecção de texto.
    pasta_saida: Pasta de saída dos arquivos.

  Returns:
    Nenhum.
  """

  # Obter as imagens da pasta
  imagens = glob.glob(f"{caminho_pasta}/*.jpg")

  # Criar a pasta de saída
  os.makedirs(pasta_saida)

  # Processar as imagens
  for imagem in imagens:
    # Criar o caminho da saída
    caminho_saida = f"{pasta_saida}/{os.path.basename(imagem)}_mascara.tif"

    # Criar a máscara
    mascaras = sam.predict(image=imagem,
                            text_prompt=texto_prompt,
                            box_threshold=box_t,
                            text_threshold=text_t,
                            output=caminho_saida)

    # Exportar a máscara e caixas delimitadoras para shapefile
    shapefile_saida = f"{pasta_saida}/{os.path.basename(imagem)}__{texto_prompt}.shp"
    boxes_saida = f"{pasta_saida}/{os.path.basename(imagem)}__{texto_prompt}_boxes.shp"

    sam.raster_to_vector(caminho_saida, shapefile_saida)
    LangSAM.save_boxes(sam, boxes_saida)

In [4]:
# Exemplo de uso

caminho_pasta = 'G:\\drone_img_sam\\praia_brava_16-12-2023\\101MEDIA_georref\\'
texto_prompt = 'vegetation'
box_t = 0.5
text_t = 0.5
pasta_saida = 'saida/processamento_lote/vegetacao0.5'

processar_imagens(caminho_pasta, texto_prompt, box_t, text_t, pasta_saida)